# Исследование надежности заемщиков

## 1. Изучаю датасет и общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/___/data.csv')
except:
    data = pd.read_csv('https://______/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных

### 2.1. Удаление пропусков

Вывожу количество пропущенных значений для каждого столбца:

In [4]:
data.isna().sum().sort_values(ascending = False)

days_employed       2174
total_income        2174
children               0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Заполню пропуски в столбце `total_income` медианным значением по каждому типу из столбца income_type.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### 2.2. Обработка аномальных значений¶

Обработаю значения в столбце `days_employed` - заменю все отрицательные значения положительными с помощью метода abs().

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведу медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median').sort_values(ascending = False)

income_type
безработный        366413.652744
пенсионер          365213.306266
в декрете            3296.759962
госслужащий          2689.368353
сотрудник            1574.202821
компаньон            1547.382223
студент               578.751554
предприниматель       520.848083
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставляю их как есть.

Выведу перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть аномальные значения. Удалю строки, в которых встречаются такие аномальные значения.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведу перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### 2.3. Удаление пропусков

Заполню пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Проверю, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.4 Изменение типов данных

Заменю вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`

In [13]:
data['total_income'] = data['total_income'].astype(int)

### 2.5 Обработка дубликатов

Обработаю неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведу их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Посмотрю количество строк-дубликатов в данных. Если такие строки присутствуют, удалю их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### 2.6 Категоризация данных

Создам следующие категории через функцию:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрю уникальные цели взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создам функцию со следующими категориями:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3. Исследование данных

### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data.groupby('children')['debt'].agg(['count', 'sum', 'mean']).round(3)

,count,sum,mean
children,,,
0,14091,1063,0.075
1,4808,444,0.092
2,2052,194,0.095
3,330,27,0.082
4,41,4,0.098
5,9,0,0.000


<div style="border:solid Chocolate 2px; padding: 20px">

**Вывод:** Из текущей таблицы мы видим, что семьи с 5-ю детьми брали кредиты, но при этом вернули их в срок, на текущий момент сложно сделать более подробные выводы по этой группе.

Самое большое число кредитов наблюдается у семей без детей, однако доля кредитов, не погашенных в срок, меньше в сравнении с остальными группами. Таким образом, можно сказать, что семьи без детей чаще берут кредиты, но при этом являлись платежеспособными и реже остальных делали просрочки по оплате долга.

Самые частые просрочки наблюдаются у семей с 1, 2-мя и 4-мя детьми, что позволяет предположить, что вероятность наличия долга увеличивается с ростом числа детей в семье, что говорит о наличии зависимости количества детей и просрочки долга. Однако, количество наблюдений у семей с 3-мя и 4-мя детьми невелико, что может повлият на общий вывод.

### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']).round(3)

,count,sum,mean
family_status,,,
Не женат / не замужем,2796,273,0.098
в разводе,1189,84,0.071
вдовец / вдова,951,63,0.066
гражданский брак,4134,385,0.093
женат / замужем,12261,927,0.076


<div style="border:solid Chocolate 2px; padding: 20px">

**Вывод:** По результатам сводной таблицы можем наблюдать, что граждане в категории "вдовец/вдова" и "в разводе" меньше всего берут кредиты, а если берут, то отдают с минимальной в сравнении с другими категориями просрочкой.

Больше всего берут кредитов семьи, зарегистрированные в браке, однако процент просрочки срока возврата кредита меньше,чем у людей, живущих в гражданском браке либо в статусе "не женат/не замужем".

Самый высокий процент по задолженности составляет у тех, кто находится в гражданском браке, либо "не женат/не замужем". В этих группах задолженность более 9%.

По результатам таблицы можно сказать, что зависимость между семейным положением и возвратом кредита в срок существует, так как мы видим, что граждане, состоящие в неформальных отношениях (без пары или в гражданском браке)ассоциируются с более высоким уровнем нестабильности, нежеланием взять на себя долгосрочные обязательства, и, соответственно, самой высокой долей просрочки по платежам. Люди, состоящие в официальном браке, или бывшие в нем ("в разводе", "вдовец/вдова") имеют более высокий уровень стабильности.

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']).round(3)

,count,sum,mean
total_income_category,,,
A,25,2,0.080
B,5014,354,0.071
C,15921,1353,0.085
D,349,21,0.060
E,22,2,0.091


<div style="border:solid blue 2px; padding: 20px">

**Категории:**
- **A** - 1000001 и выше,
- **B** - 200001–1000000,
- **C** - 50001–200000,
- **D** - 30001–50000,
- **E** - 0–30000.

<div style="border:solid Chocolate 2px; padding: 20px">

**Вывод:** Из таблицы мы видим, что самые малочисленные категории составляют граждане с доходом до 50000 рублей и от 1000000 рублей. Исходя из маленького числа наблюдений, сложно дать конкретные факты, но можно сделать предположения.

Самый высокий процент задолженности у категории с доходом до 30000 рублей -9.1% и это неудивительно, потому что эту категрию граждан можно отнести к людям с низкими доходами. Есть предположение, что их текущего дохода может не хватать даже на то, чтобы покрыть базовые потребности. Наблюдается нестабильное финансовое положение.  

Также, что удивительно, высокий процент по задолженности наблюдается у малочисленной группы граждан с очень высокими доходами - 8.0%. Можно предположить, что это связано с инвестиционными целями - богатые люди берут кредиты для вложения ивестиций, чтобы еще больше увеличить свой капитал, однако, не всегда вовремя делают погашения.

Самый низкий риск невозврата кредита в срок наблюдается у граждан с доходом ниже среднего - от 30001 до 50000 рублей, что может указывать на их большую финансовую дисциплину. По данным «СберИндекс» медианная зарплата в России составляла 52 269 рублей (лето 2023). Можно сказать, что данная категория нацелена на планирование расходов, что позволяет им покрывать базовые потребности и вовремя выплачивать долг по кредиту.

Граждане со средними доходами в диапазоне от 50001 до 200000 рублей составляют большинство и имеют чуть выше среднего уровень риска невозврата кредитов - 8.5%. Это может говорить о том, что в этой категории граждане уже могут не только покрыть свои базовые потребности, но и совершать дополнительные покупки и брать больше кредитов.

Граждане с высоким доходом (от 200001 до 1000000 рублей) имеют относительно низкий уровень просрочек по кредитам, что подчеркивает их финансовую устойчивость


*Наблюдается связь между уровнем дохода и возвратом кредита в срок*. Клиенты с очень низкими и очень высокими доходами имеют более высокую долю просроченных кредитов по сравнению с гражданами со средним и высоким уровня дохода.

### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']).round(3)

,count,sum,mean
purpose_category,,,
операции с автомобилем,4279,400,0.093
операции с недвижимостью,10751,780,0.073
получение образования,3988,369,0.093
проведение свадьбы,2313,183,0.079


<div style="border:solid Chocolate 2px; padding: 20px">

**Вывод:** Из текущей таблицы мы видим, что операции с автомобилем и получение образования имеют наибольший процент невозврата кредита в срок (9.3%). Эти цели могут быть связаны с высокими затратами на обслуживание автомобиля и образовательные услуги, а также с потенциальной нестабильностью доходов граждан.

Проведение свадьбы как категории цели составляет 7.9% невозврата долна в срок. Это может быть связано с тем, что сейчас увеличивается бюджет на проведение свадьбы, так как появились дополнительные расходы в виде кейтеринга, фотографа/видеографа, ведущего, дизайна помещения, фотосессий, повышение расходов на работу с подрядчиками (аренда помещения, организаторы свадьбы и т.д.). Однако, % невозврата меньше, чем у операций с автомобилями, что может быть связано с тем, что свадьба - запланированное событие и расходы могут быть запланированы.

Операции с недвижимостью показывают наименьший процент просрочки - 7.3%. Инвестиции в недвижимость часто рассматриваются как более надежные и долгосрочные, а также могут способствовать более ответственному подходу к финансовым обязательствам. Также, если не платить за кредитную недвижимость, в которой живет гражданин, ее могут забрать, так как до выплаты кредита недвижимость принадлежит банку, поэтому граждане внимательнее могут относится к такого рода операциям. 

В заключении моэно сказать, что *вид цели кредита влияет на вероятность его возврата в срок*. Кредиты, взятые на операции с недвижимостью, являются наиболее надежными в плане своевременного погашения по сравнению с кредитами на автомобили и образование.

## 4.Общий вывод

Основной целью проекта было **выявить, существуют ли зависимости, влияющие на факт погашения кредита в срок** (например, семейное положение, количество детей и т.д).

На основе проведенной работы можно сформулировать следующие выводы:

**1) Предобработка данных** 

*Пропущенные значения.*

В рассматриваемых данных были найдены пропущенные значения в 2-х столбцах: общий трудовой стаж в днях и ежемесячный доход. Эти пропуски могут быть в результате ошибки при сборе данных, например, из-за человческого фактора, намеренного или ненамеренного сокрытия информации, также из-за технической ошибки или поврежденного файла.
Для решения данной проблемы было использовано замещение пропущенных значений медианными, так как в данных наблюдался большой разброс в месячном доходе от 18000 рублей до 2000000 рублей. Данное решение помогло минимизировать влияние на общую статистику.

*Некорректные типы данных.*

Данные в некоторых столбцах были представлены в некорректных форматах, что могло вызвать проблемы при анализе.
Для решения данной проблемы данные были приведены к корректному формату: числовые значения, имеющие тип строковых, были преобразованы в числа.

*Дубликаты.*

В данных были обнаружены дубликаты, которые могут исказить результаты анализа. Дубликаты были удалены, чтобы обеспечить точность анализа.

*Большое количество однотипных данных, написанных по-разному.*

В данных были замечены одинаковые по логике, но разные по написанию типы данных, например, 'покупка жилья' и 'жилье', что затрудняло анализ.
Для решения проблемы данные по уровню дохода и целям кредита были категоризированы, чтобы в дальнейшем было легче найти зависимости.

**2) Цели проекта** 

В ходе работы были выявлены следующие зависимости:

*Зависимость между количеством детей и возвратом кредита в срок.*

Анализ показал, что существует определенная зависимость между количеством детей и возвратом кредита в срок, а именно, что самые частые просрочки наблюдаются у семей с 1, 2-мя и 4-мя детьми - 9.2%, 9.5% и 9.8% соответственно. Это позволяет предположить, что вероятность наличия долга увеличивается с ростом числа детей в семье, однако это зависимость не прямая, так как семьи с 5-ю детьми не имеют просрочек, а семьи с 3-мя детьми имеют меньший процент просрочек - 8.2%.

*Зависимость между семейным положением и возвратом кредита в срок.*

Результат анализа показывает зависимость между семейным положением и возвратом кредита в срок, так как граждане, состоящие в неформальных отношениях (без пары или в гражданском браке)ассоциируются с более высоким уровнем нестабильности, нежеланием взять на себя долгосрочные обязательства, и, соответственно, самой высокой долей просрочки по платежам - 9.8% и 9.3% соответственно. Люди, состоящие в официальном браке, или бывшие в нем ("в разводе", "вдовец/вдова") имеют более высокий уровень стабильности - 7.6%, 7.1%, 6.6%.

*Зависимость между уровнем дохода и возвратом кредита в срок.*

Анализ показал, что существует определенная зависимость между уровнем дохода и вероятностью возврата кредита в срок.

Самые высокорискованные группы - граждане с низким доходом до 30000 рублей (9.1%) и граждане со средним доходом в диапазоне от 50001 до 200000 рублей (8.5%). Есть предположение, что категориям с низким уровнем дохода может не хватать их текущего дохода даже на покрытие базовых потребностей, а категории со средним уровнем дохода могут не только покрыть свои базовые потребности, но и совершать дополнительные покупки и иметь несколько кредитов.

Самый низкий риск невозврата кредита в срок наблюдается у граждан с доходом ниже среднего - от 30001 до 50000 рублей, что может указывать на их большую финансовую дисциплину.

*Зависимость между целью кредита и возвратом кредита в срок.*

Было выявлено, что цель кредита также влияет на вероятность его своевременного возврата. Кредиты, взятые на операции с недвижимостью, имели наименьший процент просрочек (7.3%), в то время как кредиты на приобретение автомобиля и образование демонстрировали более высокие уровни просрочек - 9.3%.

Исходя из проведенной работы можно сказать, что финансовое поведение граждан зависит от нескольких факторов: наличие и количество детей, уровень дохода и цель кредита. На текущий момент можно сделать предположение, что наибольшая вероятность просрочки наблюдается у граждан, состоящих в официальном браке, имеющих от 1 до 3-х детей с доходом от 50000 до 200000, которые покупают в кредит машины или оплачивают образование детям.

**3) Рекомендации для заказчика** 

На основе анализа можно предложить следующие рекомендации:

*Усилить проверку граждан с уровнем дохода от 50001 до 200000 рублей.*

Учитывая наивысший процент просрочек среди граждан с доходами от 50001 до 200000 рублей, рекомендуется проводить более тщательную проверку финансового состояния и кредитной истории таких клиентов.

*Обратить внимание на цели кредита.*

При оценке заявок на кредит нужно учитывать цель кредитования. Кредиты на покупку недвижимости имеют меньший риск просрочки, что делает их более предпочтительными для выдачи.

*Создать персонализированные программы для каждой категориии граждан.*

Создание специальных кредитных продуктов, адаптированных под потребности и финансовые возможности различных категорий граждан, может помочь снизить риск просрочек.